# EasySplit

Ziyu Tang, Xinlan Wu

In [1]:
# packages for testing
import pandas as pd
import numpy as np
import ipytest
ipytest.autoconfig()
import pytest

## Object-oriented Functions

In [2]:
class BillSplit:
    """ Records of Bill Spilt
    With input activity,payer,amount, activity participants, and all participants, compute the get the records of bill split.
    """
    def __init__(self, name_list:list):
        column_names = ["Activity", "Payer", "Amount"] + name_list + ['share']
        self.df_ = pd.DataFrame(columns = column_names)
        self.name_list = name_list


    def add_act(self, Act: str, Payer: str, Amount: float, Partis: list):
        """ Add the Records of Bills
            :return: df
        """
        partis_ = [int(x) for x in [item in Partis for item in self.name_list]]
        share = Amount/sum(partis_)
        new_row = [Act, Payer, Amount] + partis_ + [share]
    
        #global Bill_df
        
        a_series = pd.Series(new_row, index = self.df_.columns)
        self.df_ = self.df_.append(a_series, ignore_index=True)
        #return df
    
    
    def balance(self):
        """ Get Balance Sheet
            :return: df
        """
        # get the spending for each
        df1 = self.df_[['Payer','Amount']].groupby('Payer').sum().reset_index()
    
        # get the amount each one should pay
        df2 = self.df_.drop(columns = ['Payer','Amount'])
        df2 = pd.melt(df2, id_vars=['Activity','share'], var_name='participants', value_name='Share')
        df2 = df2[df2.Share == 1].drop(columns = ['Share']).sort_values(['Activity','participants']).reset_index(drop=True)
        df3 = df2[['participants','share']].groupby('participants').sum().reset_index()
    
        # merge and calculate balance for each
        #global name_list
        balance_df = pd.DataFrame(self.name_list,columns=['All'])
        balance_df = pd.merge(balance_df, df1, left_on= 'All', right_on='Payer', how = 'left')
        balance_df = pd.merge(balance_df, df3, left_on= 'All', right_on='participants', how = 'left')
        balance_df = balance_df.drop(columns = ['Payer','participants']).fillna(0)
        balance_df = balance_df.assign(balance = balance_df.Amount - balance_df.share)
        return balance_df
    
    def solution(self):
        """ Get Splitting Solution
            :return: df
        """
        df = self.balance()
        if( (sum(df.balance>0) > 0) & (sum(df.balance<0) >0)):
            df = df[['All','balance']]
            # seperate payer and payee, positive balance are payee
            df_pos = df[df.balance>0].sort_values('balance', ascending =False).reset_index(drop = True)
            df_neg = df[df.balance<0].sort_values('balance').reset_index(drop = True)
        
            # while someone have uneven balance, keep making transaction
            solution_df = pd.DataFrame(columns = ['Payer','Amount','Payee'])
            while (sum(df_pos.balance) > 10e-5) | (abs(sum(df_neg.balance)) > 10e-5):
                Transaction_Amount = min([abs(df_pos.balance[0]), abs(df_neg.balance[0])])
                payer = df_neg.All[0]
                payee = df_pos.All[0]
                new_row = [payer, Transaction_Amount, payee]
                a_series = pd.Series(new_row, index = solution_df.columns)
                solution_df = solution_df.append(a_series, ignore_index=True)
                #print(payer + ' send $' + str(Transaction_Amount) + ' to ' + payee)
                
            
                # update value after above transaction
                df_pos.at[0, 'balance'] =  df_pos.balance[0] - Transaction_Amount
                df_neg.at[0, 'balance'] =  df_neg.balance[0] + Transaction_Amount

                df_pos = df_pos.sort_values('balance', ascending =False).reset_index(drop = True)
                df_neg = df_neg.sort_values('balance').reset_index(drop = True)
            df = df_pos.append(df_neg)
            return solution_df
        else:
            print('Done')
            
    def Visual(self):
        """ Get the Dataframe for Visualization
            :return: df
        """
        # get the spending for each
        df1 = self.df_[['Payer','Amount']].groupby('Payer').sum().reset_index()
    
        # get the amount each one should pay
        df2 = self.df_.drop(columns = ['Payer','Amount'])
        df2 = pd.melt(df2, id_vars=['Activity','share'], var_name='participants', value_name='Share')
        df2 = df2[df2.Share == 1].drop(columns = ['Share']).sort_values(['Activity','participants']).reset_index(drop=True)
        return df2

## Execute tests

In [3]:
%%run_pytest[clean]

@pytest.mark.parametrize('name_list',
                         [ (['a','b','c','d','e','f']) ])
def test_Initial_df(name_list):
    Bill = BillSplit(name_list)
    assert True    

@pytest.mark.parametrize(['name_list','Act', 'Payer', 'Amount', 'Partis'],
                         [ (['a','b','c','d','e','f'],'dining','a', 100.32, ['a','b','f'] ) ])
def test_add_act(name_list, Act, Payer, Amount, Partis):
    Bill = BillSplit(name_list)
    Bill.add_act(Act, Payer, Amount, Partis)
    assert True
    
@pytest.mark.parametrize(['name_list','Act', 'Payer', 'Amount', 'Partis'],
                         [ (['a','b','c','d','e','f'],'dining','a', 100.32, ['a','b','f'] ) ])
def test_balance(name_list, Act, Payer, Amount, Partis):
    Bill = BillSplit(name_list)
    Bill.add_act(Act, Payer, Amount, Partis)
    Bill.balance()
    assert True


@pytest.mark.parametrize(['name_list','Act', 'Payer', 'Amount', 'Partis'],
                         [ (['a','b','c','d','e','f'],'dining','a', 100.32, ['a','b','f'] ) ])
def test_solution(name_list, Act, Payer, Amount, Partis):
    Bill = BillSplit(name_list)
    Bill.add_act(Act, Payer, Amount, Partis)
    Bill.solution()
    assert True
    
@pytest.mark.parametrize(['name_list','Act', 'Payer', 'Amount', 'Partis'],
                         [ (['a','b','c','d','e','f'],'dining','a', 100.32, ['a','b','f'] ) ])
def test_visual(name_list, Act, Payer, Amount, Partis):
    Bill = BillSplit(name_list)
    Bill.add_act(Act, Payer, Amount, Partis)
    Bill.Visual()
    assert True

.....                                                                    [100%]
5 passed in 1.09s


## App

In [4]:
import pandas as pd
from pandas import DataFrame
import numpy as np

import dash
import dash_table
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
from dash.dependencies import Input, Output, State
from dash_extensions.snippets import send_data_frame
from dash_extensions import Download

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [5]:
#============================================== Layout ==============================================#

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MINTY])

# Create list to store the inputs
name_list = []
category = ["Dining", "Entertainment", "Housing", "Shopping", "Transportation", "Others"]
payer = []
activity = []
amount = []
participants = []
details = [0,0,0,0]

# Functions for creating visualizations
def pie_plot(df):
    large_rockwell_template = dict(
                               layout = go.Layout(title_font = dict(family = "Rockwell", size = 18))
                               )
    
    fig = px.pie(df, values = "Spending", names = "Activity",\
             color_discrete_sequence = px.colors.qualitative.Pastel,\
             template = large_rockwell_template,\
             title = "Proportion of Total Spending by Category")
    fig.update_traces(textinfo = "percent+label")
    return fig

def bar_plot(df):
    large_rockwell_template = dict(
                               layout = go.Layout(title_font = dict(family = "Rockwell", size = 18))
                               )
    
    fig = px.bar(df, x = "participants", y = "Spending",\
             color_discrete_sequence = px.colors.qualitative.Pastel,\
             template=large_rockwell_template,\
             title = "Rank of Individual Spending")
    return fig

# Function for data format
def format(x):
    return "${:.2f}".format(x)


# Input Groups of Expenses Info
controls_self = dbc.Card(
    
    [
        html.Br(),
        
        dbc.FormGroup(
                       [
                         dbc.Label("All Participants' Names"),
                         dbc.Input(id = "all-participants-name", type = "text")
                       ]
                      ),
        
        dbc.Button("Submit", outline = False, color = "primary", className = "mr-1",\
                   id = "submit-all-participant", n_clicks = 0),
        
        dbc.FormGroup(
                       [
                        dbc.Label("Payer"),
                        dcc.Dropdown(id = "payer-select"),
                       ]
                     ),
        
        dbc.FormGroup(
                       [
                        dbc.Label("Category"),
                        dcc.Dropdown(
                                      options = [{"label" : i,"value" : i}for i in category],
                                      id = "category-select"
                                    ),
                       ]
                      ),
        
        dbc.InputGroup(
                       [
                        dbc.InputGroupAddon("$", addon_type = "prepend"),
                        dbc.Input(id = "amount", placeholder = "Amount", type = "number", min = 0)
                       ],
                        className = "mb-3",
                      ),
        
        dbc.FormGroup(
                       [
                        dbc.Label("Activity Participants"),
                        dbc.Checklist(
                                      id = "activity-participant-select",
                                      inline = True
                                     ),
                       ]
                     ),
        
        
        html.Br(),
        
        dbc.Button("Add Expenses Record", outline = False, color = "primary", \
                   className = "mr-1",id = "submit-expenses-details",n_clicks = 0),
    ],
    
    body=True,
)


# Expenses Detailed Table
expenses_details = dbc.Card(
                            dbc.CardBody(
                                        [
                                         html.H5("Expenses Details",
                                         style = {
                                                 "textAlign": "left",
                                                 "fontSize": 25
                                                }
                                         ),
    
                                         html.P("Download the records:"),
    
                                         dash_table.DataTable(
                                                              id = "expenses_dt",
                                                              style_header={
                                                                            "backgroundColor": "rgb(247, 208, 119)",
                                                                            "fontWeight": "bold",
                                                                            "color": "grey"
                                                                           },
                                                              page_size = 11
                                                              ),
                                            
                                         html.Br(), 
                                        
                                         dbc.Row(
                                                  [
                                                    dbc.Button("Get Split Result", \
                                                    id = "result-button", color = "primary", className = "mb-3"),

                                                    html.Div([dbc.Button("Download Table", id = "download-button", \
                                                                         color = "warning", className = "mb-3"), \
                                                              Download(id = "download")]),
                                                  ],
                                                   align = "top",
                                                  ),
                                        ]
                                        )
                            )


# Tabs of Solution and Summary
tabs_results = dbc.Card(
                         html.Div(
                                   [
                                    dbc.Tabs(
                                            [
                                              dbc.Tab(label = "Split Result", tab_id = "tab-1"),
                                              dbc.Tab(label = "Expenses Summary", tab_id = "tab-2"),
                                              dbc.Tab(label = "Individual Expenses Summary", tab_id = "tab-3"), 
                                            ],
                                            id = "tabs",
                                            active_tab = "tab-1")
                                   ]
                                )
                       )

tab_content = dbc.Card(dbc.CardBody([html.Div(id = "content")]))


# Application layout
app.layout = dbc.Container(
                           [
                            dbc.Row(
                                    dbc.Col(
                                            html.H1("EasySplit")
                                           )
                                    ),
        
                            dbc.Row(
                                    dbc.Col(
                                            html.P("Make It Easy For Sharing Expenses \
                                            With Others"), md = 8
                                           )  
                                    ),
        
                            dbc.Row(
                                    [
                                     dcc.Store(id = "memory_all_participant"),
                                     dcc.Store(id = "memory_input"),
                                     dbc.Col(controls_self, md = 4),
                                     dbc.Col(expenses_details, md = 8)
                                    ],
                                    align = "top",
                                    ),
                            
                            html.Br(),
                               
                            #dbc.Row(tabs_results),
                            
                            dbc.Row(dbc.Col(html.Div(
                                [
                                    tabs_results,
                                    tab_content  
                                ])))
                               
                           ],
                           id = "main-container",
                           style = {"display": "flex", "flex-direction": "column"},
                           fluid = True
                           )



#============================================== Callbacks ==============================================#

# Store all names of participants to a list
@app.callback(Output("memory_all_participant", "data"),
            [Input("submit-all-participant","n_clicks")],
            [State("all-participants-name", "value")])

def save_payer_name_options(n_clicks,name):
    if n_clicks:
        name_list.append(name)
        return name_list

    
# A Dropdown for payer with the options of all people 
@app.callback(Output("payer-select", "options"),
            [Input("memory_all_participant", "data")])

def set_payer_name_options(participant_list):
    option_list = participant_list
    return [{"label": i, "value": i} for i in option_list]


# A Checkbox for activity participants with the options of all people
@app.callback(Output("activity-participant-select", "options"),
            [Input("memory_all_participant", "data")])

def set_participant_name_options(check_list):
    activity_participant = check_list
    return [{"label": i, "value": i} for i in activity_participant]


@app.callback(Output("activity-participant-select", "value"),
            [Input("memory_all_participant", "data")])

def participant_value(value_list):
    value = []
    for i in value_list:
        value.append(i)
    return value


# Show the header of expenses details table
@app.callback(Output("expenses_dt", "columns"),
            [Input("memory_all_participant", "data")])

def set_dt_header(header_data):
    Bill = BillSplit(header_data)
    dt_header = Bill.df_.columns.tolist()
    return [{"name": i, "id": i,} for i in dt_header]


# Store the data of expenses details
@app.callback(Output("memory_input", "data"),
            [Input("submit-expenses-details","n_clicks")],
            [State("category-select", "value"),
             State("payer-select", "value"),
             State("amount", "value"),
             State("activity-participant-select", "value")
            ])

def save_expenses_record(n_clicks, category, payer_name, amount_value, participants_name):
    if n_clicks:
        activity.append(category)
        payer.append(payer_name)
        amount.append(amount_value)
        participants.append(participants_name)
    
        details[0] = activity
        details[1] = payer
        details[2] = amount
        details[3] = participants
    
        return details


# Show the data of expenses details
@app.callback(Output("expenses_dt", "data"),
             [Input("memory_input", "data"),
              Input("memory_all_participant", "data")]
             )

def show_expenses_records(input_list, names):
    Bill = BillSplit(names)
    for i in range(len(input_list[1])):
        Bill.add_act(input_list[0][i], input_list[1][i], input_list[2][i], input_list[3][i])
    df = Bill.df_
    df["Amount"] = df["Amount"].apply(format)
    df["share"] = df["share"].apply(format)
    
    return df.to_dict("rows")


# Download the data of expenses details
@app.callback(Output("download", "data"),
             [Input("download-button","n_clicks")],
             [State("memory_input", "data"),
              State("memory_all_participant", "data")])

def generate_csv(n_clicks, input_list, names):
    if n_clicks:
        Bill = BillSplit(names)
        for i in range(len(input_list[1])):
            Bill.add_act(input_list[0][i], input_list[1][i], input_list[2][i], input_list[3][i])
        df = Bill.df_
        df["Amount"] = df["Amount"].apply(format)
        df["share"] = df["share"].apply(format)
        
        return send_data_frame(df.to_csv, filename = "Billing.csv")


# Result: Split Result (Tab 1), Summary Visualization (Tab 2), and Individual Visualization (Tab 3)
@app.callback(Output("content", "children"), 
              [Input("result-button","n_clicks"),
               Input("tabs", "active_tab")],
              [State("memory_input", "data"),
              State("memory_all_participant", "data")])

def switch_tab(n_clicks, at, input_list, names):
    if n_clicks:
        if at == "tab-1":
            Bill = BillSplit(names)
            for i in range(len(input_list[1])):
                Bill.add_act(input_list[0][i], input_list[1][i], input_list[2][i], input_list[3][i])
            
            solution_result = Bill.solution()
            solution_result["Amount"] = solution_result["Amount"].apply(format)
            balance_result = Bill.balance()[["All","balance"]].rename(columns = {"All" : "Name", "balance": "Balance"})
            balance_result["Balance"] = balance_result["Balance"].apply(format)
            
            return html.Div(
                [
                    html.Br(),
                    
                    
                    dbc.Row(dbc.Col(html.H5('Recommended Splitting Solution',
                                                    style = {
                                                            'textAlign': 'left',
                                                            'fontSize': 25
                                                            }))),
                    html.Br(),
                    
                    html.P('This table shows the solution of splitting the bills.'),
                    
                    html.P("Payer means the one who gives money,\
                            payee means the one who receives money."),
                    
                    html.Br(),
                    
                    dbc.Row(dbc.Table.from_dataframe(solution_result)),
                    
                    html.Br(),
                    
                    dbc.Row([dbc.Col(html.H5('Balance Sheet',
                                                    style = {
                                                            'textAlign': 'left',
                                                            'fontSize': 25
                                                            }))]),
                    html.Br(),
                    
                    html.P("This table shows the balance sheet."),
                    
                    html.P("Negative number means the amount you owe,\
                            positive number means the amount you will receive from others."),
                    
                    
                    dbc.Row(dbc.Table.from_dataframe(balance_result))
                    
                
                ])
   
    
        if at == "tab-2":
            
            Bill = BillSplit(names)
            for i in range(len(input_list[1])):
                Bill.add_act(input_list[0][i], input_list[1][i], input_list[2][i], input_list[3][i])
            
            visual = Bill.Visual()
            visual = visual.rename(columns = {"share" : "Spending"})
            visual["Spending"] = round(visual["Spending"],2)
            visual_all_pie = visual.groupby(["Activity"]).sum().reset_index()
            visual_all_bar = visual.groupby(["participants"]).sum().reset_index()
            visual_all_bar = visual_all_bar.sort_values("Spending", ascending = False)
            
            all_bar_plot = bar_plot(visual_all_bar)
            all_pie_plot = pie_plot(visual_all_pie)
            
            # plot of rank of spending

            first_card = dbc.Card(
                              [
                               dbc.CardBody(
                                            [
                                              
                                              html.Div(dbc.Row(dcc.Graph(
                                                                         figure = all_bar_plot
                                                                        )
                                                              )
                                                      )
                                            ]
                                            )
                              ]
                                )

            # plot of distribution of spending

            second_card = dbc.Card(
                              [
                               dbc.CardBody(
                                            [
                                              
                                              html.Div(dbc.Row(dcc.Graph(
                                                                         figure = all_pie_plot
                                                                        )
                                                              )
                                                      )
                                            ]
                                            )
                              ]
                                )

            return html.Div(
                          [
                           dbc.Row(html.H5("Plots of the Rank and Distribution of the Spending",
                                                    style = {
                                                            "textAlign": "left",
                                                            "fontSize": 25
                                                            })),
                           dbc.Row(html.Br()),
                           dbc.Row([dbc.Col(first_card, width = 6), dbc.Col(second_card, width = 6)])  
                          ]
                        )
        
        if at == "tab-3":
            return html.Div(
                          [
                          dbc.Row(html.H5("Choose Person to Get Individual Summary",
                                                   style = {
                                                          "textAlign": "left",
                                                          "fontSize": 25
                                                          })),
                           html.Br(),
                           
                           dbc.Row(html.H5(html.Div(id = "display-title"),
                                                   style = {
                                                          "textAlign": "left",
                                                          "fontSize": 20
                                                          })),
                           html.Br(),
                              
                           dbc.Row(
                                    [
                                     dbc.Col(dcc.Dropdown(
                                                          options = [{"label" : i,"value" : i}for i in names],
                                                          value = names[0],
                                                          id = "individual-name-select"
                                                          ), md = 4),
                                     dbc.Col(html.Div(id = "individual-summary"), md = 8)
                                    ],
                                    align = "top",
                                    ),
                          ]
                        )

        
# Interactive content of Individual Summary
@app.callback(Output("display-title", "children"),
             [Input("individual-name-select", "value")]
             )

def individual_summary_title(individual):
    return u'Spending Distribution of {}:'.format(individual)


@app.callback(Output("individual-summary", "children"),
             [Input("memory_input", "data"),
              Input("memory_all_participant", "data"),
              Input("individual-name-select", "value")]
             )

def individual_summary(input_list, names, individual):
    Bill = BillSplit(names)
    for i in range(len(input_list[1])):
        Bill.add_act(input_list[0][i], input_list[1][i], input_list[2][i], input_list[3][i])
    
    visual = Bill.Visual()
    visual = visual.rename(columns={"share" : "Spending"})
    visual["Spending"] = round(visual["Spending"],2)
    individual_pie_df = visual[visual["participants"] == individual].groupby(["Activity"]).sum().reset_index()
    individual_pie_plot = pie_plot(individual_pie_df)
    
    return dbc.Row(dcc.Graph(figure = individual_pie_plot))
        
        

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False,dev_tools_ui=False,dev_tools_props_check=False,\
                   port=8060, host='127.0.0.1')  

Dash is running on http://127.0.0.1:8060/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
